## Overview

In [14]:
import json, warnings, os
from pathlib import Path

warnings.filterwarnings('ignore')

if Path.cwd().name == "notebooks":
    rootdir = Path.cwd().parent
else:
    rootdir = Path.cwd()

import pandas as pd
import plotly.express as px
from plotly import graph_objects as go

from stables.utils.postgres import get_sqlalchemy_engine
from stables.config import local_pg_config, remote_pg_config
engine = get_sqlalchemy_engine(local_pg_config)

### 1. TVL

In [18]:
query = """
SELECT * FROM llama.circulating
"""
df= pd.read_sql(query, engine)

l = df['time'].max()

tvl_usde = df[
    (df['time'] == l) & (df['id'] == 146)
]['circulating'].sum()

tvl_usdtb = df[
    (df['time'] == l) & (df['id'] == 221)
]['circulating'].sum()

tvl = tvl_usde + tvl_usdtb


n_chains_usde = df[
    (df['time'] == l) & (df['id'] == 146)
]['chain'].nunique()
n_chains_usdtb = df[
    (df['time'] == l) & (df['id'] == 221)
]['chain'].nunique()


df = df.groupby(['time', 'id']).agg(
    tvl=('circulating', 'sum')
).reset_index().replace({146: 'USDe', 221: 'USDTB'})

fig = px.bar(df, x="time", y="tvl", color="id", title=f"Current TVL: {tvl/1e9:.2f}B, USDe: {tvl_usde/1e9:.2f}B, USDTB: {tvl_usdtb/1e9:.2f}B",labels="")
fig.update_layout(
    xaxis_title="Date",
    yaxis_title="TVL (USD)",
    legend_title_text="Stablecoin"
)
fig.show()

### 2. TVL by chains

In [19]:
query = """
SELECT * FROM llama.circulating
"""
df= pd.read_sql(query, engine)

l = df['time'].max()

df = df.groupby(['time', 'chain']).agg(
    tvl=('circulating', 'sum')
).reset_index()

fig = px.bar(df, x="time", y="tvl", color="chain", title="Ethena TVL by chains")
fig.update_layout(
    xaxis_title="Date",
    yaxis_title="TVL (USD)",
    legend_title_text="Chain"
)
fig.show()

### 3. USDe staking percentage

In [46]:
query = """
SELECT * FROM llama.circulating
"""
df= pd.read_sql(query, engine)
df1 = df[df['id'] == 146].groupby(['time']).agg(tvl=('circulating', 'sum')).reset_index()
df1['date'] = df1['time'].dt.date

query = """
SELECT * FROM llama.yield_pools
WHERE pool_id = '66985a81-9c51-46ca-9977-42b4fe7bc6df'
"""
df2 = pd.read_sql(query, engine)
df2['date'] = df2['time'].dt.date

df = df1.merge(df2[['date', 'tvl_usd']], on='date', how='outer').drop(columns=['time'])#, suffixes=('_usde', '_sus)
df.rename(columns={'tvl_usd': 'susde', 'tvl': 'usde'}, inplace=True)
df['staking_perc'] = (df['susde'] / df['usde']) *100
current_staking_perc = df[df['date'] == df['date'].max()]['staking_perc'].values[0]


fig = go.Figure()
# fig.add_trace(go.Scatter(x=df2['date'], y=df2['tvl_usd'], mode='lines', name='sUSDe'))
fig.add_trace(go.Bar(x=df1['date'], y=df1['tvl'], name='USDe', opacity=0.5))

fig.add_trace(go.Scatter(
      x=df['date'],
      y=df['staking_perc'],
      mode='lines',
      name='Staking %',
      yaxis='y2'
  ))
fig.update_layout(
      yaxis2=dict(
          title="%",
          overlaying='y',
          side='right'
      ),
    title="Current USDe TVL: {:.2f}B, staking {:.2f}%".format(tvl_usde/1e9, current_staking_perc),
    xaxis_title="Date",
    yaxis_title="TVL (USD)",
  )



### 4. Revenue

In [49]:
query = """
SELECT * FROM llama.protocol_revenue
"""
df = pd.read_sql(query, engine)
df['revenue_cumulative'] = df['revenue'].cumsum()

px.line(df, x="time", y="revenue_cumulative", color="protocol")

### 4. ENA
1. ENA total supply

In [63]:
query = """
SELECT price FROM llama.token_price
WHERE time = (SELECT MAX(time) FROM llama.token_price)
AND symbol = 'ENA'
LIMIT 1
"""
df = pd.read_sql(query, engine)
current_price = df['price'].iloc[0] if not df.empty else None
ena_total_supply = 15e9

full_market_cap = ena_total_supply * current_price
print(f"ENA Total Supply: {ena_total_supply}")
print(f"Current Price: {current_price}")
print(f"Full Market Cap: {full_market_cap/1e9:.2f}B")

ENA Total Supply: 15000000000.0
Current Price: 0.488445
Full Market Cap: 7.33B
